In [2]:
# Huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# Langchain
from langchain.llms import HuggingFacePipeline
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever, ContextualCompressionRetriever, ParentDocumentRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore

# Langchain Huggingface
from langchain_huggingface import HuggingFaceEmbeddings

# Langchain Community
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

# Langchain Core
from langchain_core.output_parsers import StrOutputParser

# Langchain Teddynote
from langchain_teddynote.retrievers import KiwiBM25Retriever

# Other
import torch
import numpy as np
from numpy.linalg import norm
import os
import fitz
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from langchain_core.documents import Document

/home/user/yhkim/env/RAG_yhkim/lib/python3.10/site-packages/langchain_teddynote/retrievers/__init__.py:14: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .kiwi_bm25 import KiwiBM25Retriever


# Load PDF

In [3]:
##original code: https://dacon.io/competitions/official/236295/codeshare/11616?page=1&dtype=recent
path = "/home/user/yhkim/data/train_source"         #pdf 파일이 있는 폴더 경로

pdfs = [os.path.join(path, i) for i in os.listdir(path)]


doc_texts = []
for pdf_path in pdfs:
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text()
    doc_texts.append(
        Document(
            page_content=text,
            metadata={"source": pdf_path},
        ))
        
        
        # {pdf_path: text})

# Chunking

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", model_kwargs={"device": "cuda"})
dimension_size = len(embeddings.embed_query(""))

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, chunk_overlap=50
)
doc_texts = text_splitter.create_documents(texts=[i.page_content for i in doc_texts], metadatas=[i.metadata for i in doc_texts])

## Using faiss index

In [7]:
db = FAISS.from_documents(doc_texts, embedding=embeddings, distance_strategy = DistanceStrategy.MAX_INNER_PRODUCT, normalize_L2 = True)
print(db.distance_strategy)
db.similarity_search_with_score("Give me Example", k=10)

DistanceStrategy.MAX_INNER_PRODUCT


[(Document(metadata={'source': '/home/user/yhkim/data/train_source/중소벤처기업부_창업사업화지원.pdf'}, page_content='<사업추진 절차>\n①사업공고\n➜\n②신청\n➜\n③선정평가\n➜\n④협약체결\n중소벤처기업부\n창업기업→전담기관\n평가위원회\n전담기관&주관기관\n\uf000\n⑥사후관리\n\uf000\n⑤창업 및 사업화\n중소벤처기업부&주관기관\n예비창업자ㆍ창업ㆍ벤처기업&주관기관\n* 사업별로다소차이가있을수있음'),
  0.37569994),
 (Document(metadata={'source': '/home/user/yhkim/data/train_source/중소벤처기업부_창업사업화지원.pdf'}, page_content='ㅇ 추진경위\n- 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 \n‘기술혁신형 창업기업 지원사업(현, 예비창업패키지)’ 신설(’18)\n- 창업기업 성장단계별 지원체계 구축을 위하여 예비, 초기, 도약 단계의 사업화 \n지원사업을 하나의 세부사업으로 통폐합(’19~)\n-\n제2벤처붐 확산 전략 등에 따라 4차산업혁명 분야(시스템반도체, 바이오헬스, 미\n래차) 및 비대면 분야 등을 중점 육성산업으로 선정하여 특화 지원(’20~)\n- 국정과제(32번, 예비창업부터 글로벌 유니콘까지 완결형 벤처생태계 구현) 및 새정부\n경제정책방향(신산업육성)에 따라 신산업 분야 글로벌 유망 스타트업을 발굴‧육성\n하는 ‘초격차 스타트업 1000+ 프로젝트’(‘23~‘27년) 추진\n- 국정과제(32번, 예비창업부터 글로벌 유니콘까지 완결형 벤처생태계 구현) 이행\n을 위해 대학발 창업 및 신산업 중심의 첨단기술‧딥테크 창업 활성화(’22~)\n6. 주요내용\nㅇ 사업기간 : ‘09~\nㅇ 사업규모 : 377,656백만원 / 창업기업 3,500개사 지원\nㅇ 사업시행방법 : 직접, 보조, 출연\nㅇ 사업시행주체 : 중소벤처기업부, 창업진흥원 등\nㅇ 사업

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype = "bfloat16",
    device_map = 'auto',
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    top_k=5,
    top_p=0.95,
    return_full_text=False,
    max_new_tokens=128,

)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [11]:
df = pd.read_csv(path[:-7]+".csv")

In [13]:
documents={}
retrievers={}
for i in range(len(pdfs)):
    documents[pdfs[i]] = FAISS.from_documents([doc for doc in doc_texts if doc.metadata["source"] == pdfs[i]], embedding=embeddings)
    retrievers[pdfs[i]] = documents[pdfs[i]].as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 8})
### 두 dict 모두 키로 pdf 경로를, value로는 FAISS와 retriever를 가짐

In [14]:
print(documents[pdfs[0]])
print(retrievers[pdfs[0]])

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fa188ee3220> search_type='mmr' search_kwargs={'k': 3, 'fetch_k': 8}


In [16]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

In [ ]:
# ({"context": retriever | format_docs, "question": RunnablePassthrough()}| prompt | llm).invoke(question)

In [ ]:
# DataFrame의 각 행에 대해 처리
results = []    
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    # source = normalize_string(row['Source'])  #이런 잡스러운건 전부 주석으로
    source = (row['Source'])
    source = os.path.join(path, source+".pdf",)
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    # normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    # normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    # normalized_keys = {(k): v for k, v in db.items()}
    # normalized_keys = {(k): v for k, v in db.items()}

    retriever = retrievers[source]

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/496 [00:00<?, ?it/s]

Question: 2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Answering Questions:   0%|          | 1/496 [00:10<1:27:46, 10.64s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer:  2024년 중앙정부 재정체계는 다음과 같이 구성되어 있습니다.
     1. 재정체계의 구성: 
     - 재정체계는 중앙정부의 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재정적, 재

Question: 2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?


Answering Questions:   0%|          | 2/496 [00:16<1:06:15,  8.05s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer:  2024년 중앙정부의 예산 지출은 다음과 같이 구성되어 있습니다.
     1. 인건비( Personnel Expenses )
     2. 소비자물품비( Consumption of Goods and Services )
     3. 재정비( Financial Expenses )
     4. 재정수입비( Financial Receipts )
     5. 기타비( Other Expenses )
     6. 기타수입비( Other Receipts )
     7. 재정비 및 재정수입비의 차이( Financial Surplus )
     8. 재정비 및 재정수입비

Question: 기금이 예산과 다른 점은?


Answering Questions:   0%|          | 2/496 [00:19<1:19:04,  9.60s/it]
